# lcall - List Call Utilities

The `lcall` function provides powerful list mapping with optional input and output processing. It's the foundation for applying transformations to collections with fine-grained control over:

**Core Features:**
- **Flexible Mapping**: Apply any callable to list elements
- **Input Processing**: Flatten, deduplicate, filter nulls before mapping
- **Output Processing**: Flatten, deduplicate, filter nulls after mapping
- **Error Handling**: Clear validation and error propagation
- **Interruption Support**: Graceful handling of InterruptedError

In [1]:
from lionherd_core.ln import lcall

## 1. Basic Usage

Apply a function to each element in an iterable.

In [2]:
# Simple mapping
numbers = [1, 2, 3, 4, 5]
doubled = lcall(numbers, lambda x: x * 2)
print(f"Doubled: {doubled}")


# With additional arguments
def add(x, y):
    return x + y


result = lcall([1, 2, 3], add, 10)
print(f"Add 10: {result}")


# With keyword arguments
def power(x, exp=2):
    return x**exp


cubed = lcall([2, 3, 4], power, exp=3)
print(f"Cubed: {cubed}")

Doubled: [2, 4, 6, 8, 10]
Add 10: [11, 12, 13]
Cubed: [8, 27, 64]


## 2. Input Processing

Transform input before applying the function.

In [3]:
# input_flatten - flatten nested structures
nested = [[1, 2], [3, [4, 5]], 6]
flat_doubled = lcall(nested, lambda x: x * 2, input_flatten=True)
print(f"Flattened then doubled: {flat_doubled}")

# input_dropna - remove None values
with_nones = [1, None, 2, None, 3]
without_nones = lcall(with_nones, lambda x: x * 2, input_dropna=True)
print(f"Nones dropped: {without_nones}")

# input_unique - remove duplicates
duplicates = [1, 2, 2, 3, 3, 3, 4]
unique_squared = lcall(duplicates, lambda x: x**2, input_unique=True)
print(f"Unique inputs squared: {unique_squared}")

Flattened then doubled: [2, 4, 6, 8, 10, 12]
Nones dropped: [2, 4, 6]
Unique inputs squared: [1, 4, 4, 9, 9, 9, 16]


In [4]:
# Combine input processing options
messy_data = [[1, None, 2], [2, 3], None, [3, 4, 4]]
clean_result = lcall(
    messy_data,
    lambda x: x * 10,
    input_flatten=True,
    input_dropna=True,
    input_unique=True,
)
print(f"Flattened, cleaned, unique: {clean_result}")

Flattened, cleaned, unique: [10, 20, 30, 40]


## 3. Output Processing

Transform results after applying the function.

In [5]:
# output_flatten - flatten nested results
def split_range(x):
    return list(range(x))


nested_ranges = lcall([2, 3, 2], split_range)
print(f"Nested ranges: {nested_ranges}")

flat_ranges = lcall([2, 3, 2], split_range, output_flatten=True)
print(f"Flattened ranges: {flat_ranges}")

Nested ranges: [[0, 1], [0, 1, 2], [0, 1]]
Flattened ranges: [0, 1, 0, 1, 2, 0, 1]


In [6]:
# output_dropna - remove None results
def safe_divide(x):
    return 10 / x if x != 0 else None


with_nones = lcall([2, 0, 5, 0, 10], safe_divide)
print(f"With None results: {with_nones}")

without_nones = lcall([2, 0, 5, 0, 10], safe_divide, output_dropna=True)
print(f"Nones dropped from output: {without_nones}")

With None results: [5.0, None, 2.0, None, 1.0]
Nones dropped from output: [5.0, 2.0, 1.0]


In [7]:
# output_unique - remove duplicate results
def modulo(x):
    return x % 3


with_dupes = lcall([1, 2, 3, 4, 5, 6, 7], modulo)
print(f"With duplicates: {with_dupes}")

# output_unique requires flatten or dropna
unique_results = lcall(
    [1, 2, 3, 4, 5, 6, 7],
    modulo,
    output_flatten=True,
    output_unique=True,
)
print(f"Unique results: {unique_results}")

With duplicates: [1, 2, 0, 1, 2, 0, 1]
Unique results: [1, 2, 0]


## 4. Combining Input and Output Processing

Chain transformations for powerful data pipelines.

In [8]:
# Complex pipeline: nested input → flatten → process → flatten output → unique
def get_factors(n):
    """Return factors of n."""
    return [i for i in range(1, n + 1) if n % i == 0]


nested_numbers = [[6, 8], [12, None], 15]

# Get all unique factors from all numbers
all_factors = lcall(
    nested_numbers,
    get_factors,
    input_flatten=True,
    input_dropna=True,
    output_flatten=True,
    output_unique=True,
)
print(f"All unique factors: {sorted(all_factors)}")

All unique factors: [1, 2, 3, 4, 5, 6, 8, 12, 15]


In [9]:
# String processing pipeline
def extract_words(text):
    """Split text into words."""
    return text.lower().split() if text else None


sentences = [
    "Hello World",
    None,
    ["Python is", "great"],
    "Python rocks",
]

unique_words = lcall(
    sentences,
    extract_words,
    input_flatten=True,
    input_dropna=True,
    output_flatten=True,
    output_dropna=True,
    output_unique=True,
)
print(f"Unique words: {sorted(unique_words)}")

Unique words: ['great', 'hello', 'is', 'python', 'rocks', 'world']


## 5. Validation and Error Handling

lcall validates inputs and provides clear error messages.

In [10]:
# Function must be callable
try:
    lcall([1, 2, 3], "not a function")
except ValueError as e:
    print(f"✓ Rejects non-callable: {e}")

# Can accept single-element iterable of callable
result = lcall([1, 2, 3], [lambda x: x * 2])
print(f"✓ Accepts [callable]: {result}")

# But rejects multiple callables
try:
    lcall([1, 2, 3], [lambda x: x * 2, lambda x: x * 3])
except ValueError as e:
    print(f"✓ Rejects multiple callables: {e}")

✓ Rejects non-callable: func must contain exactly one callable function.
✓ Accepts [callable]: [2, 4, 6]
✓ Rejects multiple callables: func must contain exactly one callable function.


In [11]:
# output_unique requires flatten or dropna
try:
    lcall([1, 2, 3], lambda x: x % 2, output_unique=True)
except ValueError as e:
    print(f"✓ Validates output_unique requirements: {e}")

# Works with output_flatten
result = lcall([1, 2, 3], lambda x: x % 2, output_flatten=True, output_unique=True)
print(f"✓ Works with output_flatten: {result}")

✓ Validates output_unique requirements: output_unique requires output_flatten or output_dropna.
✓ Works with output_flatten: [1, 0]


In [12]:
# Exceptions propagate from function
def failing_func(x):
    if x == 3:
        raise ValueError(f"Don't like {x}")
    return x * 2


try:
    lcall([1, 2, 3, 4], failing_func)
except ValueError as e:
    print(f"✓ Propagates function exceptions: {e}")

✓ Propagates function exceptions: Don't like 3


## 6. Advanced Patterns

Sophisticated use cases and edge cases.

In [13]:
# Single value input (not iterable)
single_result = lcall(42, lambda x: x * 2)
print(f"Single value wrapped: {single_result}")

# String is iterable but treated as single value without flatten
string_result = lcall("hello", lambda x: x.upper())
print(f"String as single value: {string_result}")

# With flatten, string is split into characters
chars_result = lcall("hello", lambda x: x.upper(), input_flatten=True)
print(f"String flattened to chars: {chars_result}")

Single value wrapped: [84]
String as single value: ['H', 'E', 'L', 'L', 'O']
String flattened to chars: ['HELLO']


In [14]:
# Empty input handling
empty_result = lcall([], lambda x: x * 2)
print(f"Empty input: {empty_result}")

# All values filtered out
all_filtered = lcall([None, None, None], lambda x: x * 2, input_dropna=True)
print(f"All values filtered: {all_filtered}")

Empty input: []
All values filtered: []


In [15]:
# Tuple and set flattening
nested_with_tuples = [(1, 2), {3, 4}, [5, 6]]

# Default: tuples/sets not flattened
default_flatten = lcall(
    nested_with_tuples,
    lambda x: f"type={type(x).__name__}",
    input_flatten=True,
)
print(f"Default flatten (tuples/sets kept): {default_flatten}")

# With flatten_tuple_set: fully flattened
full_flatten = lcall(
    nested_with_tuples,
    lambda x: x * 10,
    input_flatten=True,
    input_flatten_tuple_set=True,
)
print(f"Full flatten (tuples/sets flattened): {sorted(full_flatten)}")

Default flatten (tuples/sets kept): ['type=tuple', 'type=set', 'type=int', 'type=int']
Full flatten (tuples/sets flattened): [10, 20, 30, 40, 50, 60]


In [16]:
# Working with mappings - use_values extracts dict values
users = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25},
]

# Without use_values - processes whole dicts
dict_types = lcall(users, lambda x: type(x).__name__)
print(f"Without use_values: {dict_types}")

# With use_values - extracts values from each dict
all_values = lcall(
    users,
    lambda x: x,
    input_use_values=True,
    input_flatten=True,
)
print(f"With use_values (all dict values): {all_values}")

Without use_values: ['dict', 'dict']
With use_values (all dict values): [{'name': 'Alice', 'age': 30}, {'name': 'Bob', 'age': 25}]


## 7. Performance Optimization

Understanding when input/output processing helps.

In [17]:
# input_unique prevents redundant computation
def expensive_computation(x):
    """Simulate expensive operation."""
    return sum(range(x * 1000))


data_with_dupes = [5, 10, 5, 15, 10, 5]  # Many duplicates

# Without unique: computes 6 times
result_all = lcall(data_with_dupes, expensive_computation)
print(f"Computed {len(result_all)} times: {result_all[:3]}...")

# With unique: computes only 3 times
result_unique = lcall(data_with_dupes, expensive_computation, input_unique=True)
print(f"Computed {len(result_unique)} times (unique): {result_unique}")

Computed 6 times: [12497500, 49995000, 12497500]...
Computed 6 times (unique): [12497500, 49995000, 12497500, 112492500, 49995000, 12497500]


In [18]:
# Chaining lcall for multi-stage pipelines
data = [[1, 2, 3], [4, 5], [6, 7, 8, 9]]

# Stage 1: Flatten and get lengths
stage1 = lcall(data, len)
print(f"Stage 1 (lengths): {stage1}")

# Stage 2: Double the lengths
stage2 = lcall(stage1, lambda x: x * 2)
print(f"Stage 2 (doubled): {stage2}")

# Or as a single pipeline
pipeline_result = lcall(
    data,
    lambda sublist: len(sublist) * 2,
)
print(f"Pipeline (single pass): {pipeline_result}")

Stage 1 (lengths): [3, 2, 4]
Stage 2 (doubled): [6, 4, 8]
Pipeline (single pass): [6, 4, 8]


## Summary Checklist

**lcall Essentials:**
- ✅ Apply any callable to iterable elements
- ✅ Input processing: flatten, dropna, unique, use_values
- ✅ Output processing: flatten, dropna, unique
- ✅ Pass additional args/kwargs to function
- ✅ Validation: callable check, output_unique requirements
- ✅ Single values automatically wrapped in list
- ✅ Exception propagation from function
- ✅ InterruptedError support for graceful cancellation
- ✅ Flexible tuple/set flattening control

**Common Patterns:**
- 🔄 Data cleaning: `input_flatten + input_dropna + input_unique`
- 🎯 Result deduplication: `output_flatten + output_unique`
- 🚀 Performance: `input_unique` avoids redundant computation
- 🔗 Pipelines: Chain lcall or combine with output → input processing

**Next Steps:**
- See `to_list` for understanding input/output processing details
- See async variants for concurrent operations
- Explore mapping patterns in lionherd workflows